## Calculate freeze-thaw days for NA-CORDEX

This notebook calculates annual freeze-thaw days for the NA-CORDEX dataset and saves the results in NetCDF format.

In [1]:
import xarray as xr
import numpy as np

from matplotlib.backends.backend_pdf import PdfPages
import matplotlib.pyplot as plt
from pathlib import Path
import os

## The timeit magic doesn't play well with Dask, so we need to time things by hand
# import time

import pprint
import json

### Use Dask to Speed up Computations

In [2]:
import dask
from ncar_jobqueue import NCARCluster

# Processes is processes PER CORE.
# This one works fine.

## zarr store chunks are around 200 MB, so they should fit easily into the per-memory allocation
## could probably reduce it a lot if all I'm doing is crunching, not plotting
## but you do need a fair amount for overhead of task graph, data buffering, etc.

num_jobs = 30
walltime = "4:00:00"
cluster = NCARCluster(cores=num_jobs, processes=1, memory='90GB', project='P86850057', walltime=walltime)
cluster.scale(jobs=num_jobs)

from distributed import Client
from distributed.utils import format_bytes
client = Client(cluster)
cluster

/glade/work/mcginnis/zarr/miniconda3/envs/pangeo-cordex/lib/python3.8/site-packages/distributed/node.py:151: UserWarning: Port 8787 is already in use.
Perhaps you already have a cluster running?
Hosting the HTTP server on port 45054 instead
  warnings.warn(


## Calculate freeze-thaw days from tmin & tmax

Different sources use different thresholds for defining freeze-thaw days, so these are adjustable parameters defaulting to -1 and 0 C.

In [3]:
## todo: add units parameter & check units attribute on inputs

def freezethaw(tmin_var, tmax_var, freezetemp=-1, thawtemp=0):
    '''Calculate freeze-thaw days where tmin < freeze threshold and tmax > thawing threshold'''
    result = (tmin_var < freezetemp) & (tmax_var > thawtemp)
    return result
    

## Read in data from zarr stores

In [4]:
indir  = '/glade/scratch/bonnland/na-cordex/zarr-publish/'
outdir = '/glade/scratch/mcginnis/deloitte/canada/zarr/'


tmin_ds = xr.open_zarr(indir+"tmin.hist-rcp85.day.NAM-44i.raw.zarr", consolidated=True)
tmax_ds = xr.open_zarr(indir+"tmax.hist-rcp85.day.NAM-44i.raw.zarr", consolidated=True)


# p = Path(indir)
# stores = list(p.rglob("*.zarr"))
# stores = list(p.rglob("tmax.rcp85.day.NAM-44i.raw.zarr"))
# stores = list(p.rglob("uas.rcp85.*.zarr"))
# for store in stores:
#     print(f'Opening {store}...')
#     try:
#         ds = xr.open_zarr(store.as_posix(), consolidated=True)
#         print('\n')
#     except Exception as e:
#         print(e)
#         continue

#    data_vars = [vname for vname in ds.data_vars]
#    data_var = data_vars[0]
#    store_name = store.as_posix().split('/')[-1]


In [5]:
# Check that data cube looks reasonable

tmax_ds

<xarray.Dataset>
Dimensions:    (bnds: 2, lat: 129, lon: 300, member_id: 15, time: 55517)
Coordinates:
  * lat        (lat) float64 12.25 12.75 13.25 13.75 ... 74.75 75.25 75.75 76.25
  * lon        (lon) float64 -171.8 -171.2 -170.8 ... -23.25 -22.75 -22.25
  * member_id  (member_id) <U21 'MPI-ESM-MR.CRCM5-UQAM' ... 'MPI-ESM-LR.WRF'
  * time       (time) datetime64[ns] 1949-01-01T12:00:00 ... 2100-12-31T12:00:00
    time_bnds  (time, bnds) datetime64[ns] dask.array<chunksize=(10410, 2), meta=np.ndarray>
Dimensions without coordinates: bnds
Data variables:
    tmax       (member_id, time, lat, lon) float32 dask.array<chunksize=(4, 1000, 65, 150), meta=np.ndarray>
Attributes:
    CORDEX_domain:                  NAM-44
    contact:                        {"MPI-ESM-MR.CRCM5-UQAM": "Winger.Katja@u...
    contact_note:                   {"HadGEM2-ES.RegCM4": "Simulations by Ray...
    creation_date:                  {"hist": {"MPI-ESM-MR.CRCM5-UQAM": "2018-...
    driving_experiment:             {"hist": {"MPI-ESM-MR.CRCM5-UQAM": "MPI-M...
    driving_experiment_name:        {"hist": "historical", "rcp85": "rcp85"}
    driving_model_ensemble_member:  {"MPI-ESM-MR.CRCM5-UQAM": "r1i1p1", "GEMa...
    driving_model_id:               {"MPI-ESM-MR.CRCM5-UQAM": "MPI-M-MPI-ESM-...
    experiment_id:                  {"hist": "historical", "rcp85": "rcp85"}
    frequency:                      day
    id:                             doi:10.5065/D6SJ1JCH
    institute_id:                   {"MPI-ESM-MR.CRCM5-UQAM": "UQAM", "GEMatm...
    institution:                    {"MPI-ESM-MR.CRCM5-UQAM": "Universite du ...
    model_id:                       {"MPI-ESM-MR.CRCM5-UQAM": "UQAM-CRCM5", "...
    product:                        output
    project_id:                     CORDEX
    rcm_version_id:                 {"MPI-ESM-MR.CRCM5-UQAM": "v1", "GEMatm-C...
    references:                     {"MPI-ESM-MR.CRCM5-UQAM": "http://www.mrc...
    title:                          {"hist": {"MPI-ESM-MR.CRCM5-UQAM": "NA-CO...
    tracking_id:                    {"hist": {"MPI-ESM-MR.CRCM5-UQAM": "80a8d...
    version:                        {"hist": {"MPI-ESM-MR.CRCM5-UQAM": "3.1",...
    zarr-dataset-reference:         For dataset documentation, see DOI https:...
    zarr-note-time:                 Historical data runs 1950 to 2005, future...
    zarr-version:                   1.0

In [6]:
## This step only constructs the task graph.
## The computation doesn't happen until explicitly called.
ft = freezethaw(tmin_ds["tmin"],tmax_ds["tmax"])

In [7]:
ft

,Array,Chunk
Bytes,32.23 GB,39.00 MB
Shape,"(15, 55517, 129, 300)","(4, 1000, 65, 150)"
Count,4482 Tasks,896 Chunks
Type,bool,numpy.ndarray


In [8]:
## The timeit magic doesn't play nicely with Dask, so we time by hand instead.
## Hmm... I thought.  But even this is locking up.  So maybe it's something else.
## Try again later.


# start = time.time()

ft.compute()

# finish = time.time()
# print(finish - start)

<xarray.DataArray (member_id: 15, time: 55517, lat: 129, lon: 300)>
array([[[[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
...
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]],

        [[False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         ...,
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False],
         [False, False, False, ..., False, False, False]]]])
Coordinates:
  * lat        (lat) float64 12.25 12.75 13.25 13.75 ... 74.75 75.25 75.75 76.25
  * lon        (lon) float64 -171.8 -171.2 -170.8 ... -23.25 -22.75 -22.25
  * member_id  (member_id) <U21 'MPI-ESM-MR.CRCM5-UQAM' ... 'MPI-ESM-LR.WRF'
  * time       (time) datetime64[ns] 1949-01-01T12:00:00 ... 2100-12-31T12:00:00

### Plot a single timestep to check that it looks reasonable

In [ ]:
## Todo: clean up excess code

def plotMap(ax, map_slice, date_object=None, member_id=None):
    '''Create a map plot on the given axes, with min/max as text'''

    ax.imshow(map_slice, origin='lower')

    minval = map_slice.min(dim = ['lat', 'lon'])
    maxval = map_slice.max(dim = ['lat', 'lon'])

    # Format values to have at least 4 digits of precision.
    ax.text(0.01, 0.03, "%4g" % minval, transform=ax.transAxes, fontsize=12)
    ax.text(0.99, 0.03, "%4g" % maxval, transform=ax.transAxes, fontsize=12, horizontalalignment='right')
    ax.set_xticks([])
    ax.set_yticks([])
    
    if date_object:
        ax.set_title(date_object.values.astype(str)[:10], fontsize=12)
        
    if member_id:
        ax.set_ylabel(member_id, fontsize=12)
        
    return ax

In [ ]:
sim = 'MPI-ESM-LR.RegCM4'
date = "1950-01-01"

slice = ft.sel(member_id=sim, time=date).squeeze()

fig = plt.figure()
ax = plt.gca()

plotMap(ax, slice)


## Aggregate to yearly totals

In [ ]:
aft = ft.groupby('time.year').sum('time')

In [ ]:
aft

In [ ]:
aft.compute()

### plot timestep to see if it looks reasonable

## write to netcdf

In [ ]:
# xarrayobject.to_netcdf("filepath")
# look at brian's zarrify notebook for metadata examples

### Release the workers.

In [ ]:
!date

In [9]:
cluster.close()

distributed.client - ERROR - Failed to reconnect to scheduler after 10.00 seconds, closing client
_GatheringFuture exception was never retrieved
future: <_GatheringFuture finished exception=CancelledError()>
asyncio.exceptions.CancelledError


#### Create Single Map Plot (Helper Function)

### Create Time Series Plots over Multiple Pages
These also mark the locations of missing values.

In [ ]:
def plot_timeseries(ds, data_var, store_name, plotdir):
    # Generate plot. 
    #
    # With 30 workers, expect 1 minute walltime for computation and 1-2 minutes for plot rendering.
    #
    member_names = ds.coords['member_id'].values
    numEnsembleMembers = member_names.size

    numPlotsPerPage = 4
    numPages = np.ceil(numEnsembleMembers / numPlotsPerPage).astype(int)
    numPlotCols = 1

    figWidth = 25 
    figHeight = 20

    linewidth = 0.5

    pp = PdfPages(f'{plotdir}/{store_name}_ts.pdf')

    for pageNum in range(numPages):

        memberStart = pageNum * numPlotsPerPage
        memberEnd = np.min((memberStart + numPlotsPerPage, numEnsembleMembers))
        plot_row_index = 0
        
        # If this is the final page, target empty subplots for hiding.
        removeBlankSubplots = (pageNum == numPages-1) and (numEnsembleMembers < memberStart + numPlotsPerPage)

        # Plot the aggregate statistics across time.
        fig, axs = plt.subplots(numPlotsPerPage, numPlotCols, figsize=(figWidth, figHeight))
        #fig, axs = plt.subplots(numPlotsPerPage, numPlotCols, figsize=(figWidth, figHeight), sharey='col')

        print(f'Shape of subplots object: {axs.shape}')
        
        
        for index in np.arange(memberStart, memberEnd):
            mem_id = member_names[index]
            data_slice = ds[data_var].sel(member_id=mem_id)
            unit_string = ds[data_var].attrs['units']
            
            min_vals = data_slice.min(dim = ['lat', 'lon'])
            max_vals = data_slice.max(dim = ['lat', 'lon'])
            mean_vals = data_slice.mean(dim = ['lat', 'lon'])
            std_vals = data_slice.std(dim = ['lat', 'lon'])

            nan_indexes = np.isnan(min_vals)
            nan_times = ds.time[nan_indexes]

            axs[plot_row_index].plot(ds.time, min_vals, linewidth=linewidth, label='min')
            axs[plot_row_index].plot(ds.time, max_vals, linewidth=linewidth, label='max')
            axs[plot_row_index].plot(ds.time, mean_vals, linewidth=linewidth, label='mean')
            axs[plot_row_index].plot(ds.time, std_vals, linewidth=linewidth, label='std')
            
            ymin, ymax = axs[plot_row_index].get_ylim()
            rug_y = ymin + 0.01*(ymax-ymin)
            axs[plot_row_index].plot(nan_times, [rug_y]*len(nan_times), '|', color='m', label='isnan')
            axs[plot_row_index].set_title(mem_id, fontsize=20)
            axs[plot_row_index].legend(loc='upper right')
            axs[plot_row_index].set_ylabel(unit_string)

            plot_row_index = plot_row_index + 1

        plt.suptitle(store_name, fontsize=25)
        plt.tight_layout(pad=10.2, w_pad=3.5, h_pad=3.5)
        
        if removeBlankSubplots:
            hide_subplots(axs, plot_row_index)
            
        pp.savefig()
        plt.close()
    pp.close()


#### Function Producing Maps of First, Middle, Last Timesteps

In [ ]:
def getValidDateIndexes(member_slice):
    '''Search for the first and last dates with finite values.'''
    min_values = member_slice.min(dim = ['lat', 'lon'])
    is_finite = np.isfinite(min_values)
    finite_indexes = np.where(is_finite)
    start_index = finite_indexes[0][0]
    end_index = finite_indexes[0][-1]
    #print(f'start ={start_index}, end={end_index}')
    return start_index, end_index


def plot_first_mid_last(ds, data_var, store_name, plotdir):
    # Generate plot. 
    #
    # With 30 workers, expect 1 minute walltime for computation and 1-2 minutes for plot rendering.
    #
    member_names = ds.coords['member_id'].values
    numEnsembleMembers = member_names.size

    numPlotsPerPage = 4
    numPages = np.ceil(numEnsembleMembers / numPlotsPerPage).astype(int)
    numPlotCols = 3

    figWidth = 18 
    figHeight = 12 #20

    pp = PdfPages(f'{plotdir}/{store_name}_fml.pdf')

    for pageNum in range(numPages):

        memberStart = pageNum * numPlotsPerPage
        memberEnd = np.min((memberStart + numPlotsPerPage, numEnsembleMembers))
        plot_row_index = 0

        # If this is the final page, target empty subplots for hiding.
        removeBlankSubplots = (pageNum == numPages-1) and (numEnsembleMembers < memberStart + numPlotsPerPage)

        # Plot the aggregate statistics across time.
        fig, axs = plt.subplots(numPlotsPerPage, numPlotCols, figsize=(figWidth, figHeight), constrained_layout=True)

        for index in np.arange(memberStart, memberEnd):
            mem_id = member_names[index]
            data_slice = ds[data_var].sel(member_id=mem_id)
            
            start_index, end_index = getValidDateIndexes(data_slice)
            midDateIndex = np.floor(len(ds.time) / 2).astype(int)

            startDate = ds.time[start_index]
            first_step = data_slice.sel(time=startDate) 
            ax = axs[plot_row_index, 0]
            plotMap(ax, first_step, startDate, mem_id)

            midDate = ds.time[midDateIndex]
            mid_step = data_slice.sel(time=midDate)   
            ax = axs[plot_row_index, 1]
            plotMap(ax, mid_step, midDate)

            endDate = ds.time[end_index]
            last_step = data_slice.sel(time=endDate)            
            ax = axs[plot_row_index, 2]
            plotMap(ax, last_step, endDate)
            
            plot_row_index = plot_row_index + 1
 
        plt.suptitle(store_name, fontsize=20)

        if removeBlankSubplots:
            hide_subplots(axs, plot_row_index)
            
        pp.savefig()
        plt.close()

    pp.close()

### Loop over Zarr Stores in Directory and Make Plots.

In [ ]:
# For now, make the Zarr output directory a global variable.
#dirout = '/glade/scratch/bonnland/na-cordex/zarr-demo'
zarr_directory = '/glade/scratch/bonnland/na-cordex/zarr-publish/'
#plot_directory = '/glade/scratch/bonnland/na-cordex/zarr-plots-test/'
plot_directory = "."

p = Path(zarr_directory)
stores = list(p.rglob("*.zarr"))
#stores = list(p.rglob("uas.rcp85.*.zarr"))
stores = list(p.rglob("tmax.rcp85.day.NAM-44i.raw.zarr"))
for store in stores:
    print(f'Opening {store}...')
    try:
        ds = xr.open_zarr(store.as_posix(), consolidated=True)
        print('\n')
    except Exception as e:
        print(e)
        continue
    data_vars = [vname for vname in ds.data_vars]
    data_var = data_vars[0]
    store_name = store.as_posix().split('/')[-1]
    
    # Only produce plots that haven't been created already.  
    plotdir = plot_directory + store_name
    if not os.path.exists(plotdir):
        os.makedirs(plotdir)
    else:
        # Plots exist; skip to the next case.
        continue
    
    plot_stat_maps(ds, data_var, store_name, plotdir)
    plot_first_mid_last(ds, data_var, store_name, plotdir)
    plot_timeseries(ds, data_var, store_name, plotdir)
    

## Unused Plot Functions

### Function producing Super-Wide Time Series Plots. 

In [ ]:
def plot_timeseries_wide(ds, data_var, store_name):
    # Generate super-wide plot of individual time series. 
    #
    # With 30 workers, expect 1 minute walltime for computation and 1-2 minutes for plot rendering.
    #
    member_names = ds.coords['member_id'].values
    numEnsembleMembers = member_names.size

    numPlotRows = numEnsembleMembers
    numPlotCols = 4

    figWidth = 200 #100 
    figHeight = 80

    linestyle = 'k.'
    linewidth = 0.5
    markersize = 0.5

    fig, axs = plt.subplots(numPlotRows, numPlotCols, figsize=(figWidth,figHeight), sharey='col')

    for index in range(numEnsembleMembers):
        mem_id = member_names[index]
        data_slice = ds[data_var].sel(member_id=mem_id)

        data_agg = data_slice.min(dim=['lat', 'lon'])
        axs[index, 0].plot(ds.time, data_agg, linewidth=linewidth)
        axs[index, 0].plot(ds.time, data_agg, linestyle, markersize=markersize)
        axs[index, 0].set_ylabel(mem_id, fontsize=15)

        data_agg = data_slice.max(dim=['lat', 'lon'])
        axs[index, 1].plot(ds.time, data_agg, linewidth=linewidth)
        axs[index, 1].plot(ds.time, data_agg, linestyle, markersize=markersize)
        axs[index, 1].set_ylabel(mem_id, fontsize=15)

        data_agg = data_slice.mean(dim=['lat', 'lon'])
        axs[index, 2].plot(ds.time, data_agg, linewidth=linewidth)
        axs[index, 2].plot(ds.time, data_agg, linestyle, markersize=markersize)
        axs[index, 2].set_ylabel(mem_id, fontsize=15)

        data_agg = data_slice.std(dim=['lat', 'lon'])
        axs[index, 3].plot(ds.time, data_agg, linewidth=linewidth)
        axs[index, 3].plot(ds.time, data_agg, linestyle, markersize=markersize)
        axs[index, 3].set_ylabel(mem_id, fontsize=15)
       
    
    axs[0, 0].set_title(f'min({var})', fontsize=40)
    axs[0, 1].set_title(f'max({var})', fontsize=40)
    axs[0, 2].set_title(f'mean({var})', fontsize=40)
    axs[0, 3].set_title(f'std({var})', fontsize=40)
    

    plt.suptitle(store, fontsize=50)
    plt.tight_layout(pad=20.2, w_pad=5.5, h_pad=5.5)
    plt.savefig(f'{store_name}_ts.pdf')



#### Function Producing Map Plots Over a SINGLE Page

In [ ]:
def plot_maps_OLD(ds, data_var, store_name):
    # Generate plot. 
    #
    # With 30 workers, expect 1 minute walltime for computation and 1-2 minutes for plot rendering.
    #
    member_names = ds.coords['member_id'].values
    numEnsembleMembers = member_names.size

    numPlotsPerPage = 4
    numPages = np.ceil(numEnsembleMembers / numPlotsPerPage).astype(int)

    numPlotRows = numEnsembleMembers
    numPlotCols = 4

    figWidth = 17 
    figHeight = 35

    fig, axs = plt.subplots(numPlotRows, numPlotCols, figsize=(figWidth,figHeight), constrained_layout=True)

    for index in range(numEnsembleMembers):
        mem_id = member_names[index]
        data_slice = ds[data_var].sel(member_id=mem_id)

        data_agg = data_slice.min(dim='time')
        pcm0 = axs[index, 0].imshow(data_agg, origin='lower')
        axs[index, 0].set_ylabel(mem_id, fontsize=8)

        data_agg = data_slice.max(dim='time')
        pcm1 = axs[index, 1].imshow(data_agg, origin='lower')
        axs[index, 1].set_ylabel(mem_id, fontsize=8)

        data_agg = data_slice.mean(dim='time')
        pcm2 = axs[index, 2].imshow(data_agg, origin='lower')
        axs[index, 2].set_ylabel(mem_id, fontsize=8)

        data_agg = data_slice.std(dim='time')
        pcm3 = axs[index, 3].imshow(data_agg, origin='lower')
        axs[index, 3].set_ylabel(mem_id, fontsize=8)
       
    
    axs[0, 0].set_title(f'min({var})', fontsize=15)
    axs[0, 1].set_title(f'max({var})', fontsize=15)
    axs[0, 2].set_title(f'mean({var})', fontsize=15)
    axs[0, 3].set_title(f'std({var})', fontsize=15)
    
    plt.colorbar(pcm0, ax = axs[:, 0], location='bottom', shrink=0.7)
    plt.colorbar(pcm1, ax = axs[:, 1], location='bottom', shrink=0.7)
    plt.colorbar(pcm2, ax = axs[:, 2], location='bottom', shrink=0.7)
    plt.colorbar(pcm3, ax = axs[:, 3], location='bottom', shrink=0.7)

    plt.suptitle(store_name, fontsize=20)
    plt.savefig(f'{store_name}_maps.pdf')
